# Instance visualizer
In this notebook I'm going to eplore different ways I can visualize an instant. I'll use this to create a video visualization of continous instances one after the other

#### Some Code (ignoreable)

In [1]:
%matplotlib qt
# %matplotlib widget

In [2]:
import dataloader
import pandas as pd
import os
import numpy as np
sid = '08db4255286f'

In [3]:
import importlib
importlib.reload(dataloader)
acc_data = dataloader.acc_data_for_child(sid, True)

loaded dataframe in  2.525 seconds


In [4]:
acc_data.head()

,series_id,step,timestamp,anglez,enmo
0,08db4255286f,0,2018-11-05 14:00:00+00:00,-30.845301,0.0447
1,08db4255286f,1,2018-11-05 14:00:05+00:00,-34.181801,0.0443
2,08db4255286f,2,2018-11-05 14:00:10+00:00,-33.877102,0.0483
3,08db4255286f,3,2018-11-05 14:00:15+00:00,-34.282101,0.0680
4,08db4255286f,4,2018-11-05 14:00:20+00:00,-34.385799,0.0768


In [5]:
events = pd.read_csv(os.path.join('data', 'train_events.csv'))
# events['timestamp'] = pd.to_datetime(events['timestamp'], utc=True)
events[events['series_id']==sid].head()

,series_id,night,event,step,timestamp
398,08db4255286f,1,onset,11988.0,2018-11-06T02:39:00-0400
399,08db4255286f,1,wakeup,14388.0,2018-11-06T05:59:00-0400
400,08db4255286f,2,onset,28680.0,2018-11-07T01:50:00-0400
401,08db4255286f,2,wakeup,31320.0,2018-11-07T05:30:00-0400
402,08db4255286f,3,onset,44184.0,2018-11-07T23:22:00-0400


In [6]:
dataloader.annotate_sid(acc_data, events, sid)

In [7]:
acc_data.sample(10)

,series_id,step,timestamp,anglez,enmo,activity
343398,08db4255286f,343398,2018-11-25 10:56:30+00:00,-30.255400,0.0012,Waking
161687,08db4255286f,161687,2018-11-14 22:33:55+00:00,24.207701,0.0069,Waking
159698,08db4255286f,159698,2018-11-14 19:48:10+00:00,-52.489799,0.1019,Waking
47996,08db4255286f,47996,2018-11-08 08:39:40+00:00,-78.212402,0.0005,Sleeping
201729,08db4255286f,201729,2018-11-17 06:10:45+00:00,-75.267303,0.0089,Waking
334941,08db4255286f,334941,2018-11-24 23:11:45+00:00,-63.130501,0.0108,Waking
117867,08db4255286f,117867,2018-11-12 09:42:15+00:00,-37.117901,0.0000,Sleeping
181911,08db4255286f,181911,2018-11-16 02:39:15+00:00,-43.793999,0.0018,Sleeping
207318,08db4255286f,207318,2018-11-17 13:56:30+00:00,-15.325700,0.0240,Waking
393330,08db4255286f,393330,2018-11-28 08:17:30+00:00,-27.892599,0.0132,Sleeping


##### 20 minutes before and after a timestamp

In [ ]:
ts = pd.Timestamp('2018-11-07 05:50:00+00:00')
ts

In [ ]:
delta = pd.Timedelta(20, 'min')

In [ ]:
(ts - delta, ts + delta)

In [ ]:
idx = (acc_data['timestamp'] >= (ts-delta)) & (acc_data['timestamp'] < (ts+delta))
idx

In [ ]:
dat = acc_data.loc[idx]

In [ ]:
dat

## Visual Frame

To visualize the data at a fixed timestamp I'm taking all the data 20 minutes before and after that timestamp, then plotting their `anglez` and `enmo`. Each point is also given a color based on how many steps it is before or ahead of the timestamp. Finally I added a jitter to the cordinates to reveal points that are hidden one behind the other.

Run the next cell to see an example of a frame where the child transitioned from waking to sleeping

In [9]:
import matplotlib.pyplot as plt
from matplotlib.axes import Axes
import matplotlib.animation as anim
fig, ax = plt.subplots()
ax.scatter([1,2,3], [5,6,7])

In [ ]:
# %matplotlib inline # To show a static image in the notebook
# %matplotlib notebook # To show the interactive video using matplotlib backend
import matplotlib.pyplot as plt
from matplotlib.axes import Axes
import matplotlib.animation as anim
fig, ax = plt.subplots()
ax: Axes = ax
ax.set_xlabel('anglez')
ax.set_ylabel('enmo')
ax.set_title(f'20 minutes around {ts} for {sid}')
x = dat['anglez']
y = dat['enmo']
dat_len = len(dat)
mid = dat['step'].iat[int(dat_len/2)]
c = [i - mid for i in dat['step']]
coll = ax.scatter(x,y, c=c, cmap='plasma', marker='o')
plt.colorbar(coll, ax=ax)
txt = ax.text(-10,0.15,'Some Text')
x_jit = (np.random.random_sample((dat_len,))-0.5)*x.std()
y_jit = (np.random.random_sample((dat_len,))-0.5)*y.std()
jit = [i/100.0 for i in range(30)]
# print(jit)
def animate(frame):
    j = jit[frame]
    coll.set(offsets=[i for i in zip(x+x_jit*j, y+y_jit*j)])
    txt.set_text(f'Jitter factor = {j}')
    return [coll, txt]
fig.tight_layout()
ani = anim.FuncAnimation(func=animate, fig=fig, frames=len(jit), blit=True)
plt.show()

In [8]:
import matplotlib.pyplot as plt
from matplotlib.axes import Axes
import matplotlib.animation as anim
from matplotlib.collections import PathCollection
def init_frame(ax: Axes, data: pd.DataFrame):
    '''
    Create an empty figure with containers for everything that will go in it.

    Returns
    -------
    A list of the artists
    '''
    points = ax.scatter(x=[], y=[], c=[], cmap='plasma')
    ax.set_xlabel('anglez')
    ax.set_ylabel('enmo')
    xmin = data['anglez'].min()
    xmax = data['anglez'].max()
    buff = (xmax-xmin)*0.1
    ax.set_xlim(xmin-buff, xmax+buff)
    ymin = data['enmo'].min()
    ymax = data['enmo'].max()
    buff = (ymax-ymin)*0.1
    ax.set_ylim(ymin-buff, ymax+buff)
    activity_text = ax.text((xmax+xmin)/2, ymax-data['enmo'].std(), "SOME TEXT", ha='center', weight='bold', va='bottom')
    return [points, activity_text]

def draw_frame(ts: pd.Timestamp, data: pd.DataFrame, window: pd.Timedelta, x_jit_scale: int, y_jit_scale: int, points: PathCollection, activity_text: plt.Text):
    artists = []
    if type(ts) is not pd.Timestamp:
        ts = pd.Timestamp(ts)
    idx = (data['timestamp'] >= ts-window) & (data['timestamp'] < ts+window)
    data = data.loc[idx, ['step', 'anglez', 'enmo', 'activity']]
    # Set X, Y points
    x = data['anglez']
    y = data['enmo']
    dat_len = len(data)
    mid = data['step'].iat[int(dat_len/2)]
    c = [i - mid for i in data['step']]
    x_jit = (np.random.random_sample(x.shape)-0.5)*x_jit_scale
    y_jit = (np.random.random_sample(y.shape)-0.5)*y_jit_scale
    jit = 0.3
    points.set(offsets=list(zip(x+x_jit*jit,y+y_jit*jit)), array=c)
    artists.append(points)
    # Set the text box
    txt = f"{ts}\n{data['activity'].iat[0]} => {data['activity'].iat[-1]}"
    activity_text.set_text(txt)
    artists.append(activity_text)
    # if txt != activity_text.get_text():
        # activity_text.set_text(txt)
    #     artists.append(activity_text)
    return artists

In [9]:
from tqdm.notebook import tqdm

fig, ax = plt.subplots()
artists = init_frame(ax, acc_data)
ani = anim.FuncAnimation(
    fig=fig, 
    func=draw_frame, 
    frames=tqdm(acc_data['timestamp'][::60]), 
    fargs=(acc_data, pd.Timedelta(20, 'min'), acc_data['anglez'].std(), 2*acc_data['enmo'].std(), *artists),
    blit=True,
    interval=100
)
# plt.show()

  0%|          | 0/7338 [00:00<?, ?it/s]

In [ ]:
ani.save?

In [ ]:
ani.save(os.path.join('vids', 'activity3.mp4'))